# Phase 4: Execution (Review & Verification)

- **Goal**: 작성된 챕터별 강의 자료의 품질을 검증하고, Finalized Brief(기획안)와의 일치 여부를 확인하여 최종 완성도를 보장한다.
- **Core Concept**: 전체 문서를 입력받아 **Reviewer Agent**가 환각(Hallucination), 논리적 오류, 스타일 준수 여부를 종합적으로 평가하며, 기준 미달 시 수정(Rewrite) 과정을 거쳐 통과된 문서만을 다음 단계로 넘긴다.
- **Data Flow**:
    - **Input**: `Chapter Content List` (Phase 3 산출물), `Finalized Brief` (검증 기준)
    - **Output**: `Verified Content` (검증 및 수정이 완료된 최종 본문)

## 1. Environment & API Key Setup

In [ ]:
import os
import json
import time
import sys
import google.generativeai as genai
from google.generativeai.types import content_types
from collections import defaultdict

# TODO: API Key를 설정하세요
GOOGLE_API_KEY = "" 
genai.configure(api_key=GOOGLE_API_KEY)

## 2. Input Data Placeholders
별도의 변수로 Phase 3의 결과물(Markdown List)과 Finalized Brief(JSON)를 입력받습니다.
아래 변수들에 데이터를 복사해서 넣어주세요.

In [ ]:
# Phase 3 산출물 (List of Markdown Strings) - JSON 형식의 문자열로 입력 (예: '["Chapter 1 content...", "Chapter 2 content..."]')
phase3_results_input = [] # 이는 Phase3의 final_results에 해당한다

# Finalized Brief (Phase 2 산출물 JSON)
finalized_brief_input =  # Final Breif 전체여야 함

## 3. System Prompts & JSON Schemas

In [8]:
# --- Review Agent ---

REVIEW_SYSTEM_PROMPT = """
You are the **Review Agent** for the Lecture Material Generation System.
Your goal is to evaluate the `Chapter Content` against the `Finalized Brief`.

### **Task Guidelines**
1. **Analyze Constraints**: Check if the content follows the 'Style Guide' (Tone, Formatting) and 'Chapter Objectives'.
2. **Determine Pass/Fail**:
   - **Pass**: If the content meets all criteria -> Set `is_pass: true`, others null.
   - **Fail**: If there are issues -> Set `is_pass: false`.
3. **Generate Editor Prompt**:
   - If Fail, write a specific, actionable instruction for an AI Editor to fix the issue.
   - The prompt must be self-contained and executable.

### **Few-shot Examples**

#### **Case 1: Pass (Perfect Match)**
**Input Context**:
- **Brief**: Tone="Friendly/Polite", Math="Minimize equations".
- **Content**: "안녕하세요! 오늘은 PPO에 대해 알아볼게요..." (Correct tone, no complex math)
**Output**:
{
  "is_pass": true,
  "reasoning": null,
  "editor_prompt": null
}

#### **Case 2: Fail (Tone Mismatch)**
**Input Context**:
- **Brief**: Tone="Polite(존댓말)", Target="Beginners".
- **Content**: "강화학습은 어렵다. 그냥 외워라. 식은 다음과 같다..." (Rude/Blunt tone)
**Output**:
{
  "is_pass": false,
  "reasoning": "The content uses a blunt, informal tone ('해라체') instead of the requested polite tone ('존댓말').",
  "editor_prompt": "현재 텍스트의 어조가 스타일 가이드에 맞지 않습니다. 모든 문장을 독자를 존중하는 '친절한 존댓말(해요체)'로 변환하세요. 내용은 유지하되, 어미를 부드럽게 수정하십시오."
}

#### **Case 3: Fail (Formatting Issue)**
**Input Context**:
- **Brief**: Formatting="Use bold for key terms".
- **Content**: "Neural Networks are composed of layers. Input layer receives data..." (No bolding)
**Output**:
{
  "is_pass": false,
  "reasoning": "Key terms are not highlighted as required by the formatting guide.",
  "editor_prompt": "텍스트에서 'Neural Networks', 'Input layer' 등 핵심 기술 용어를 찾아 Markdown Bold(** **) 처리를 적용하십시오. 다른 내용은 변경하지 마십시오."
}
"""

review_schema = {
  "type": "object",
  "properties": {
    "is_pass": {
      "type": "boolean",
      "description": "품질 기준 통과 여부. (True: 통과, False: 수정 필요)"
    },
    "reasoning": {
      "type": "string",
      "nullable": True,
      "description": "판정 근거. 실패 시 구체적인 문제점을 기술."
    },
    "editor_prompt": {
      "type": "string",
      "nullable": True,
      "description": "False일 경우, Editor Agent에게 전달할 구체적인 수정 지시어(Prompt). True일 경우 null."
    }
  },
  "required": ["is_pass"]
}

# --- Editor Agent ---

EDITOR_SYSTEM_PROMPT = """
You are the **Editor Agent** for the Lecture Material Generation System.
Your goal is to rewrite the provided `original_markdown` strictly following the `editor_prompt`.

### **Input Data Context**
You will receive a JSON object containing:
1. **original_markdown**: The lecture content drafted by the **Write Agent**. It follows a specific format (e.g., `### Chapter ID. Title - SubID) Name`).
2. **editor_prompt**: A specific instruction generated by the **Review Agent** based on the project's `Finalized Brief` (e.g., Tone adjustment, Formatting fixes).

### **Execution Rules (STRICT)**
- **Scope of Change**: Apply changes **ONLY** where requested by the `editor_prompt`. Preserve the rest of the content (Headers, Code blocks, Logic) exactly as is.
- **Output Format**: Return **ONLY** the corrected Markdown text. Do not include JSON wrappers or conversational fillers.
- **Header Preservation**: Never alter the top-level header format (`### ...`) unless explicitly asked.

### **Few-shot Examples**

#### **Case 1: Tone Correction (Brief requires 'Polite/Friendly' tone)**
**Input**:
{
  "original_markdown": "### 1. 인공지능 개요 - 1) AI의 역사\n인공지능은 1950년대에 시작되었다. 튜링 테스트가 그 시초다.\n#### 1. 태동기\n초기에는 기호주의(Symbolism)가 주류였다. 규칙을 일일이 입력해야 했다.",
  "editor_prompt": "현재 문서의 어조가 '평어체(해라체)'입니다. 이를 독자에게 친근하게 다가가는 '존댓말(해요체)'로 전체 문장을 수정하십시오."
}

**Output**:
### 1. 인공지능 개요 - 1) AI의 역사
인공지능은 1950년대에 시작되었어요. 튜링 테스트가 그 시초랍니다.
#### 1. 태동기
초기에는 기호주의(Symbolism)가 주류였어요. 규칙을 일일이 입력해야 했답니다.

#### **Case 2: Formatting Fix (Brief requires 'Bold for Key Terms')**
**Input**:
{
  "original_markdown": "### 3. 딥러닝 기초 - 2) 활성화 함수\n활성화 함수는 입력 신호의 총합을 출력 신호로 변환하는 함수다. 대표적으로 Sigmoid, ReLU가 있다.\n> **Note**: Sigmoid는 기울기 소실 문제가 있다.",
  "editor_prompt": "본문 내의 핵심 용어(활성화 함수, 입력 신호, 출력 신호, Sigmoid, ReLU)에 Markdown Bold(** **) 처리를 적용하십시오."
}

**Output**:
### 3. 딥러닝 기초 - 2) 활성화 함수
**활성화 함수**는 **입력 신호**의 총합을 **출력 신호**로 변환하는 함수입니다. 대표적으로 **Sigmoid**, **ReLU**가 있습니다.
> **Note**: **Sigmoid**는 기울기 소실 문제가 있습니다.

#### **Case 3: Structure/Content Adjustment (Brief requires 'Specific Example')**
**Input**:
{
  "original_markdown": "### 2. 데이터 전처리 - 3) 스케일링\n데이터의 단위를 맞추는 과정입니다.\n- Min-Max Scaler: 0과 1 사이로 변환\n- Standard Scaler: 평균 0, 분산 1로 변환",
  "editor_prompt": "설명이 너무 추상적입니다. 'Standard Scaler' 설명 하단에 '키(cm)와 몸무게(kg)'를 예시로 든 1줄 설명을 추가하십시오."
}

**Output**:
### 2. 데이터 전처리 - 3) 스케일링
데이터의 단위를 맞추는 과정입니다.
- Min-Max Scaler: 0과 1 사이로 변환
- Standard Scaler: 평균 0, 분산 1로 변환
  (예: 키 170cm와 몸무게 60kg처럼 단위가 다른 데이터를 동일한 정규 분포로 맞출 때 사용해요.)
"""

## 4. Agent Implementation

In [9]:
class ReviewerAgent:
    def __init__(self, model_name="gemini-3-flash-preview"):
        self.model = genai.GenerativeModel(
            model_name=model_name,
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": review_schema
            },
            system_instruction=REVIEW_SYSTEM_PROMPT
        )

    def review(self, chapter_content, finalized_brief):
        prompt = f"""
        [Content to Review]
        {chapter_content}

        [Finalized Brief (Criteria)]
        {json.dumps(finalized_brief, ensure_ascii=False)}
        """
        try:
            response = self.model.generate_content(prompt)
            return json.loads(response.text)
        except Exception as e:
            print(f"[Review Error] {e}")
            return None

class EditorAgent:
    def __init__(self, model_name="gemini-3-flash-preview"):
        self.model = genai.GenerativeModel(
            model_name=model_name,
            system_instruction=EDITOR_SYSTEM_PROMPT
        )

    def rewrite(self, original_markdown, editor_prompt):
        input_data = {
            "original_markdown": original_markdown,
            "editor_prompt": editor_prompt
        }
        prompt = f"""
        [Input Data]
        {json.dumps(input_data, ensure_ascii=False)}
        """
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"[Editor Error] {e}")
            return original_markdown # Fail-safe: return original if edit fails

## 5. Execution Logic

In [10]:
def Execute_Phase4(chapter_content_list, finalized_brief):
    if not chapter_content_list or not finalized_brief:
        print("Error: Missing input data.")
        return []

    # 1. 초기화
    reviewer = ReviewerAgent()
    editor = EditorAgent()
    verified_content = []
    
    print(f"Starting Phase 4 Verification for {len(chapter_content_list)} chapters...")
    
    # 2. 챕터별 검증 루프
    for i, chapter in enumerate(chapter_content_list):
        print(f"\n[Checking Chapter {i+1} / {len(chapter_content_list)}]...")
        
        # Reviewer Agent에게 검증 요청
        review_result = reviewer.review(chapter, finalized_brief)
        
        if not review_result:
            print("  > Review failed (API Error). Keeping original.")
            verified_content.append(chapter)
            continue
            
        # Case A: 통과 (Pass)
        if review_result.get('is_pass'):
            print("  > Pass (✅)")
            verified_content.append(chapter)
            
        # Case B: 실패 (Fail) - 수정 필요
        else:
            reason = review_result.get('reasoning', 'No reason provided')
            edit_prompt = review_result.get('editor_prompt', '')
            print(f"  > Fail (❌) - Reason: {reason}")
            print(f"  > Rewriting content...")

            # Editor Agent가 피드백을 반영하여 수정본 생성
            revised_chapter = editor.rewrite(chapter, edit_prompt)
            verified_content.append(revised_chapter)
            print("  > Rewrite Complete.")
            
    # 3. 최종 결과 반환
    return verified_content

## 6. Main Execution

In [11]:
if __name__ == "__main__":
    # 1. Input parsing
    try:
        if phase3_results_input.strip() and phase3_results_input.strip() != "[]":
            chapter_list = json.loads(phase3_results_input)
        else:
            chapter_list = []
            
        if finalized_brief_input.strip() and finalized_brief_input.strip() != "{}":
            final_brief_obj = json.loads(finalized_brief_input)
        else:
            final_brief_obj = None
            
    except json.JSONDecodeError as e:
        print(f"Input JSON Error: {e}")
        chapter_list = []
        final_brief_obj = None

    # 2. Validation & Execution
    if not GOOGLE_API_KEY:
        print("STOP: 1번 셀에 GOOGLE_API_KEY를 설정해주세요.")
    elif not chapter_list:
        print("STOP: Phase 3 결과물(chapter_list)을 입력해주세요.")
    elif not final_brief_obj:
        print("STOP: Phase 2 결과물(finalized_brief)을 입력해주세요.")
    else:
        # Execute
        final_verified_content = Execute_Phase4(chapter_list, final_brief_obj)
        
        # Save Trigger
        if final_verified_content:
            output_filename = "final_verified_lecture_notes.md"
            with open(output_filename, "w", encoding="utf-8") as f:
                for content in final_verified_content:
                    f.write(content + "\n\n---\n\n")
            print(f"\n[System] All verified content saved to '{output_filename}'.")

AttributeError: 'list' object has no attribute 'strip'